In [ ]:
import pandas as pd
from nd2reader import ND2Reader
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import colorcet as cc
from matplotlib.collections import LineCollection
from skimage.measure import label
from skimage.morphology import dilation, erosion, disk
import os

In [ ]:
exp_dir = '/gpfs/home/denesl01/scratch/projects/MSC_GEMs/20240206_GEM_plating_collagen_cell_density' 

In [ ]:
def read_nd2(file_path):
    with ND2Reader(file_path) as images:
        return np.array(images)

In [ ]:
def load_images(exp_dir, d):
   path = os.path.join(exp_dir, d, 'file_list.csv')
   df = pd.read_csv(path, index_col=0).reset_index(drop=True)
   for i, row in df.iterrows():
      gempath = os.path.join(exp_dir, d, 'rawdata', row['gem_file'])
      nucimpath = os.path.join(exp_dir, d, 'rawdata', row['nuc_file'])
      nucpath = os.path.join(exp_dir, d, 'png_masks', row['nuc_file'].split('.')[0] + '_cp_masks.png')
      trackspath = os.path.join(exp_dir, d, 'tracks', row['gem_file'].split('.')[0] + '_tracks.csv')
      print('opening images for', row['gem_file'] + '...')
      gem = read_nd2(gempath)
      nucim = read_nd2(nucimpath) 
      nuc = sk.io.imread(nucpath)
      print('loading tracks for', row['gem_file'] + '...')
      tracks = pd.read_csv(trackspath)
      return gem, nuc, nucim, tracks



In [ ]:
exp_list = [f for f in os.listdir(exp_dir) if '.' not in f]
gem, nuc, nucim, tracks = load_images(exp_dir, exp_list[0])
cmap = cc.cm['glasbey_hv']
tracks_nuc = tracks[tracks['compartment'] == 'nuc']


tracks_nuc = tracks_nuc.groupby('particle').filter(lambda x: len(x) > 9)

labels = sorted(tracks_nuc['nucleus'].unique())
color_dict = {label: cmap(i / len(labels)) for i, label in enumerate(labels)}
nuc_props=sk.measure.regionprops(nuc)

fig,ax = plt.subplots(1,3, figsize= (20, 10))

boundaries = sk.segmentation.find_boundaries(nuc)
boundaries_rgba = np.zeros((boundaries.shape[0], boundaries.shape[1], 4))  # Initialize with zeros
boundaries_rgba[boundaries] = [0, 0, 0, 1]
ax[0].imshow(boundaries_rgba)

gem = sk.img_as_uint(sk.filters.gaussian(gem, 1))
ax[1].imshow(np.amax(gem, axis=0,), cmap='Greys', interpolation='none', vmin=90, vmax=120)

ax[1].imshow(boundaries_rgba, interpolation='none')
ax[2].imshow(nucim[0,:,:], cmap='Greys', interpolation='none')
for label, group in tracks_nuc.groupby('nucleus'):
    num_tracks = group['particle'].nunique()
    for axs in ax:
        axs.text(nuc_props[label-1].centroid[1], nuc_props[label-1].centroid[0], num_tracks, color=color_dict[label], fontsize=10)
    for track, group in group.groupby('particle'):
        ax[0].plot(group['x'], group['y'], color=color_dict[label], lw=0.5)
        
fig.tight_layout()
plt.savefig('labeled_tracks.pdf')